In [1]:
import setup 
setup.init_django()

In [2]:
from market.models import StockQuote

In [3]:
from django.db.models import Avg, F, RowRange, Window
from django.utils import timezone
from datetime import timedelta
from decimal import Decimal

In [4]:
days_ago = 30
now = timezone.now()
start_date = now - timedelta(days=30)
end_date = now

qs = StockQuote.objects.filter(time__range=(start_date, end_date))
qs.count()

6400

In [5]:
total = 0
for obj in qs:
    total += obj.close_price

In [6]:
total / qs.count()

Decimal('416.247672765625')

In [7]:
qs.values('company').annotate(avg_price=Avg('close_price'))

<QuerySet [{'company': 1, 'avg_price': Decimal('605.9644865537848606')}, {'company': 2, 'avg_price': Decimal('247.5856175324675325')}]>

In [8]:
count = 5
ticker = "AAPL"
rolling_qs = list(qs.filter(company__ticker=ticker).order_by('-time')[:count])
rolling_qs.reverse()

In [9]:
total = 0
for i, obj in enumerate(rolling_qs):
    total += obj.close_price
    avg = total / (i + 1)
    print(i + 1, obj.id, obj.close_price, avg)

1 17791 233.7600 233.7600
2 17792 233.8100 233.7850
3 17793 233.7400 233.7700
4 17794 233.6936 233.7509
5 17795 233.7800 233.75672


In [10]:
qs = StockQuote.objects.filter(company__ticker=ticker, time__range=(start_date, end_date))

In [11]:
frame_start = -(count - 1)
ma_val = qs.annotate(
    ma=Window(
        expression=Avg('close_price'),
        order_by=F('time').asc(),
        frame=RowRange(start=frame_start, end=0),
    )
).order_by('-time')

In [12]:
for obj in ma_val[:5]:
    print(obj.id, obj.close_price, obj.ma, obj.time)

17795 233.7800 233.7567200000000000 2025-01-15 00:55:00+00:00
17794 233.6936 233.7367200000000000 2025-01-15 00:50:00+00:00
17793 233.7400 233.7080000000000000 2025-01-15 00:45:00+00:00
17792 233.8100 233.6930600000000000 2025-01-15 00:40:00+00:00
17791 233.7600 233.6410600000000000 2025-01-15 00:35:00+00:00


In [13]:
frame_start = -(count - 1)
ma_vals = qs.annotate(
    ma_5=Window(
        expression=Avg('close_price'),
        order_by=F('time').asc(),
        frame=RowRange(start=-4, end=0),
    ),
    ma_20=Window(
        expression=Avg('close_price'),
        order_by=F('time').asc(),
        frame=RowRange(start=-19, end=0),
    )
).order_by('-time').first()

ma_vals.id, ma_vals.close_price, ma_vals.ma_5, ma_vals.ma_20

(17795,
 Decimal('233.7800'),
 Decimal('233.7567200000000000'),
 Decimal('233.5385700000000000'))